# HYPER-PARAMETER TUNING - KERAS TUNER

in this notebook i use keras tuner to optimize my hyperparameters for my CNN model

## Imports

***general imports***

In [1]:
import numpy as np

***tensorflow imports***

In [2]:
!pip install keras_tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.6/169.6 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.1 MB/s eta 0:00:00


In [3]:
import tensorflow as tf 
from tensorflow import keras
import keras_tuner
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-3-0cf5315ead2b>:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


## Load Data

***import fashion_mnist dataset***

In [4]:
fashion_mnist = keras.datasets.fashion_mnist

## Pre-Process Data

***test/train split dataset***

In [5]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


***normalize images***

In [6]:
X_train = X_train/255.0
X_test  = X_test/255.0

***re-shape images***

In [7]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

## Build Model Architecture

***build main model (Convolution layer two times + flatten + relu + softmax)***

In [8]:
def build_model(hp):  

  # model structure
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=90, step=14),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=50, step=15),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=120, step=17),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  # compile our model
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model


## Hyper-Parameter Tuning

***use random search***

In [9]:
tuner_search = RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=4,directory='output3',
                          project_name="Mnist Fashion")

***result of the random searth***

In [10]:
tuner_search.search(X_train,y_train,epochs=3,validation_split=0.1)

Trial 4 Complete [00h 00m 42s]
val_accuracy: 0.8755000233650208

Best val_accuracy So Far: 0.906000018119812
Total elapsed time: 00h 02m 34s


***getting the best model***

In [11]:
model=tuner_search.get_best_models(num_models=1)[0]

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 74)        1924      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 32)        21344     
                                                                 
 flatten (Flatten)           (None, 15488)             0         
                                                                 
 dense (Dense)               (None, 117)               1812213   
                                                                 
 dense_1 (Dense)             (None, 10)                1180      
                                                                 
Total params: 1,836,661
Trainable params: 1,836,661
Non-trainable params: 0
_________________________________________________________________


## Train model

***re-train our best model***

In [11]:
model.fit(X_train, y_train, epochs=11, validation_split=0.1, initial_epoch=4)

## Evaluate Model Performance

***evalute final performance***

In [12]:
model.evaluate(X_test, y_test )

313/313 [==============================] - 1s 4ms/step - loss: 0.2760 - accuracy: 0.9038


[0.27604591846466064, 0.9038000106811523]